In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer_DeepseekR1_8B = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-8B")
model_DeepseekR1_8B = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-8B", device_map="auto",load_in_8bit=True)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': 'model/meta-llama/Llama-3.2-3B-Instruct'. Use `repo_type` argument if needed.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import json
import numpy as np
import heapq
from matplotlib import colors
import matplotlib.pyplot as plt
import torch
import networkx as nx
from networkx.readwrite import json_graph

tokenizer_Llama_32_3B = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
model_Llama_32_3B = AutoModelForCausalLM.from_pretrained("models/meta-llama/Llama-3.2-3B-Instruct", device_map="auto",load_in_8bit=True)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/opt/conda/envs/LLM/lib/python3.13/site-packages/transformers/quantizers/auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


In [3]:
import os
import json
import numpy as np
import heapq
from matplotlib import colors
import matplotlib.pyplot as plt
import torch
import networkx as nx
from networkx.readwrite import json_graph

In [4]:
with open("graphs/sejong_bus.json", "r") as json_file:
    j = json.load(json_file)

G = json_graph.node_link_graph(j)
nodes = G.nodes
edges = G.edges
stations = list(G.nodes())
heuristic_table = {station: np.random.randint(0, 10) for station in stations}

from graph_to_prompt import graph_to_prompt

/opt/conda/envs/LLM/lib/python3.13/site-packages/networkx/readwrite/json_graph/node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


In [5]:
start, goal = 6802360, 6806306
n_points = 2

prompt = graph_to_prompt(j, start, goal, n_points)
# with open('prompt.txt', 'r') as f:
#     prompt = f.read()

In [6]:
inputs = tokenizer_Llama_32_3B(
    prompt,
    return_tensors="pt",
    return_attention_mask=True
)
input_ids = inputs["input_ids"]
# 만약 pad_token_id가 없으면, eos_token_id를 패드로 지정
inputs["input_ids"] = inputs["input_ids"].to(model_Llama_32_3B.device)
inputs["attention_mask"] = inputs["attention_mask"].to(model_Llama_32_3B.device)
model_Llama_32_3B.config.pad_token_id = model_Llama_32_3B.config.eos_token_id

outputs = model_Llama_32_3B.generate(
    **inputs
)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [8]:
generated_token = outputs[:, input_ids.shape[-1] :]
result = tokenizer_Llama_32_3B.batch_decode(generated_token, skip_special_tokens=True)
print(result)

['[6802360, 6802361]']
